<a href="http://www.cosmostat.org/" target="_blank"><img align="left" width="300" src="http://www.cosmostat.org/wp-content/uploads/2017/07/CosmoStat-Logo_WhiteBK-e1499155861666.png" alt="CosmoStat Logo"></a>
<br>
<br>
<br>
<br>

# The Anatomy of a Python Class (Part II)
---

> Author: <a href="http://www.cosmostat.org/people/sfarrens" target="_blank" style="text-decoration:none; color: #F08080">Samuel Farrens</a>  
> Email: <a href="mailto:samuel.farrens@cea.fr" style="text-decoration:none; color: #F08080">samuel.farrens@cea.fr</a>  
> Year: 2019  
> Version: 1.0

---
<br>

This notebook introduces some more advanced concepts in the manipulation of Python classes. Before starting you should make sure you have completed the [first notebook](./Classes_I.ipynb) or are at least familiar with the topics covered therein. As with the first part, this tutorial is in no way exhaustive and you are encouraged to suppliment your understanding with further reading.

If you are new to Jupyter notebooks note that cells are executed by pressing <kbd>SHIFT</kbd>+<kbd>ENTER</kbd> (&#x21E7;+ &#x23ce;). See the <a href="https://jupyter-notebook.readthedocs.io/en/stable/" target_="blanck">Jupyter documentation</a> for more details.



## Contents

---

1. [Set-Up](#1-Set-Up)
1. [Inheritance](#2-Inheritance)
  1. [Inherting Attributes](#Inherting-Attributes)
  1. [Overriding](#Overriding)
  1. [Handling Instantiation](#Handling-Instantiation)
  1. [Multiple Parents](#Multiple-Parents)
  1. [Method Resolution Order](#Method-Resolution-Order)
1. [Composition](#3-Composition)
1. [Abstract Classes](#4-Abstract-Classes)
  1. [Abstract Methods](#Abstract-Methods)
  1. [Abstract Properties](#Abstract-Properties)
1. [Exercises](#5-Exercises)


## 1 Set-Up

The following cell contains some set-up commands. Be sure to execute this cell before continuing.

In [ ]:
# Notebook Set-Up Commands

def print_error(error):
    """ Print Error
    
    Function to print exceptions in red.
    
    Parameters
    ----------
    error : string
        Error message
    
    """
    print('\033[1;31m{}\033[1;m'.format(error))

## 1 Inheritance

---

A powerful property of classes is the ability to inherit attributes and methods from other classes. This helps avoid repetition of code, which makes it easier to develop and maintain. 

<br>

### Inherting Attributes

We will begin by defining a simple class with a single static method that will act as a *parent* class.

In [ ]:
# Define a parent class
class Parent:
    
    @staticmethod
    def add(a, b):
        return a + b

# Print the parent dictionary
print('Parent.__dict__ =', Parent.__dict__)

Note that this class has no special properties, it the same as classes we saw in the previous notebook.

Now we will define a second class, also with a single static function. This class will serve as a *child* class that will inherit from the parent class. This is done by simply passing the parent class name in `()` when defining the child class. 

In [ ]:
# Define a child class
class Child(Parent):
    
    @staticmethod
    def subtract(a, b):
        return a - b

print('Child.__dict__ =', Child.__dict__)

We can see that the contents of the child class dictionary are only what is expected from a normal class definition. Using the special `__bases__` attribute, however, we can see a tuple of classes that the child inherits from.

In [ ]:
print('Child.__bases__ =', Child.__bases__)

Finally, we can demonstrate that the child has indeed inherited new attributes from the parent as follows.

In [ ]:
print('1 + 2 =', Child.add(1, 2))
print('3 - 2 =', Child.subtract(3, 2))

We can clearly see that `Child` has inhertited the static `add` method from `Parent`.

In the following example, we can see that this works for any type of class attribute.

In [ ]:
class Parent:
    
    x = 1
    
class Child(Parent):
    
    @classmethod
    def show(cls):
        return print('x =', cls.x)

Child.show()

### Overriding

We saw in the previous notebook that class intance attributes will override class attributes of the same name. The same happens with child attributes with respect to parent attributes. 

In [ ]:
class Parent:
    
    x = 1
    
class Child(Parent):
    
    x = 2
    
    @classmethod
    def show(cls):
        return print('x =', cls.x)

Child.show()

### Hierachy

It is possible to define a hierchy of parent classes, the attributes of which will be inheried by a given child class.

In [ ]:
class GrandParent:
    
    x = 1
    name = 'Grandparent'

class Parent(GrandParent):
    
    y = 2
    name = 'Parent'
    
class Child(Parent):
    
    z = 3
    name = 'Child'

print(Child.name, Child.x, Child.y, Child.z)

Note that overriding will also act hierachically, meaning that the last class in the chain will be given precedence.

<br>

### Handling Instantiation 

If the parent class contains an `__init__` method, this too can be inherited by the child.

In [ ]:
class Parent:
    
    def __init__(self, value):
        self.myattr = value
    
class Child(Parent):
    
    def show(self):
        return print('myattr =', self.myattr)
    
Child('A string').show()

In fact it also works the other way around.

In [ ]:
class Parent:
    
    def show(self):
        return print('myattr =', self.myattr)
    
class Child(Parent):
    
    def __init__(self, value):
        self.myattr = value
    
Child('A string').show()

This is because the child will lookup any attributes not listed in its own dictionary in all the parent (or base) classes.

If both classes have an `__init__` method only the child class will be initialised (see [Overriding](#Overriding)).

In [ ]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
    
inst = Child()
print('cval =', inst.cval)

In [ ]:
try:
    print('pval =', inst.pval)
except Exception as error:
    print_error(error)

We can fix this by manually instantiating the parent class.

In [ ]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
        Parent.__init__(self)
    
inst = Child()
print('cval =', inst.cval)
print('pval =', inst.pval)

Python has a useful shortcut to make this process easier using the [`super`](https://docs.python.org/3/library/functions.html?highlight=super#super)function. This allows us to instantiate the parent without naming it, this particularly useful if the name of the parent class were to change.

In [ ]:
class Parent:
    
    def __init__(self):
        self.pval = 'parent value'
    
class Child(Parent):
    
    def __init__(self):
        self.cval = 'child value'
        super().__init__()
    
inst = Child()
print('cval =', inst.cval)
print('pval =', inst.pval)

### Multiple Parents

It is possible for a child to inherit attributes from multiple parents.

In [ ]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2, value3):
        self.cval = value3
        Mother.__init__(self, value1)
        Father.__init__(self, value2)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2, 3)
inst.show_mother()
inst.show_father()
inst.show_child()

Note that the parent classes were explicitly instantiated in the previous example. Attempting the same thing using the `super` function will raise an error.

In [ ]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2):
        self.cval = value2
        super().__init__(value1)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2)

In [ ]:
try:
    inst.show_mother()
    inst.show_father()
    inst.show_child()
except Exception as error:
    print_error(error)

As we can see the `Mother` parent class was properly instantiated, but the `Father` parent class was not. The reasons for this are explained in the following subsection.

> In general, caution should be used when building a class archtecture that requires inheritance for multiple parent classes.

<br>

### Method Resolution Order

The Method Resolution Order (MRO) dictates the order in which the child class will search the base classes for a given attribute. We can see the MRO of the child class from the previous example using the `__mro__` attribute as follows.

In [ ]:
print('Child.__mro__ =', Child.__mro__)

We can see that the `Mother` class appears before the `Father` class in the MRO, hence the `super` method in `Child` instantiates `Mother`. 

In order to make the previous example work, we would need to add another `super` in the `__init__` of the `Mother` class as follows.

> Note: Don't do this in your code!

In [ ]:
class Mother:
    
    def __init__(self, value):
        self.mval = value
        super().__init__(value)
        
    def show_mother(self):
        print('mother value =', self.mval)
        
class Father:
    
    def __init__(self, value):
        self.fval = value
        
    def show_father(self):
        print('father value =', self.fval)
    
class Child(Mother, Father):
    
    def __init__(self, value1, value2):
        self.cval = value2
        super().__init__(value1)
        
    def show_child(self):
        print('child value =', self.cval)
        
inst = Child(1, 2)
inst.show_mother()
inst.show_father()
inst.show_child()

This, however, is not recommended as it adds unncessary ambiguity to the code and makes debugging extremely difficult.

<br>

## 3 Composition
---

Inhertitance is not the only way in which a class can access attributes from other classes. This comes back one of main take-away messages of this tutorial, namely that everything in Python is an object and any object can be assigned as an attribute.

This means that a class attribute can actually be an instance of another class.

In [ ]:
class Composer:
    
    def __init__(self):
        self.myattr = 'composer value'
    
class myClass:
    
    def __init__(self):
        self.comp = Composer()

inst = myClass()
print("myattr =", inst.comp.myattr)

This can particularly useful as we can pass classes or class intances to our new class without knowing anything about the other classes. 

In [ ]:
class Star:
    
    def __init__(self):
        self.whoami = 'I am a star!'
        
class Galaxy:
    
    def __init__(self):
        self.whoami = 'I am a Galaxy!'
    
class myClass:
    
    def __init__(self, composer):
        self.comp = composer()

inst1 = myClass(Star)
inst2 = myClass(Galaxy)
print("whoami =", inst1.comp.whoami)
print("whoami =", inst2.comp.whoami)

We can even achieve inheritance-like properties if we know the name of a given attribute that the composer class should have.

In [ ]:
class Star:
    
    def __init__(self):
        self.whoami = 'I am a star!'
        
class Galaxy:
    
    def __init__(self):
        self.whoami = 'I am a Galaxy!'
    
class myClass:
    
    def __init__(self, composer):
        self.whoami = composer().whoami

inst1 = myClass(Star)
inst2 = myClass(Galaxy)
print("whoami =", inst1.whoami)
print("whoami =", inst2.whoami)

> Note: This will break if the composer does not have an attribute called `whoami`.

Composition also alows preinitialised class instances to be passed.

In [ ]:
class Star:
    
    def __init__(self, mag):
        self.mag = mag
        
class Galaxy:
    
    def __init__(self, mag):
        self.mag = mag
    
class myClass:
    
    def __init__(self, composer):
        self.comp = composer
        self.mag = self.comp.mag

        
star = Star(11.05)
inst1 = myClass(star)
inst2 = myClass(Galaxy(8.2))
print("mag =", inst1.mag)
print("mag =", inst2.mag)

This can be useful, especially if we want to be able to change the original instance attributes.

In [ ]:
inst1.mag = 12.3
inst1.comp.mag = 13.1
print("inst1 mag =", inst1.mag)
print("star mag =", star.mag)

Notice how the `mag` attribute of the instance `star` changed!

## 4 Abstract Classes
---

Abstract classes are classes that contain abstract methods. So it makes sense to move right along and explain what an abstract method is.

<br>

### Abstract Methods

An abstract method is a method that is defined but not implemented. This is useful for definining parent classes that impose some conditions on the child.

In [ ]:
from abc import ABC, abstractmethod

class Parent(ABC):
    
    @abstractmethod
    def get_id(self):
        pass

print ('Parent.__dict__', Parent.__dict__)

We can see that the method `get_id` has been defined but does nothing.

We now define a child class that inherits this parent.

In [ ]:
class Child(Parent):
    
    def __init__(self):
        self.whoami = 'I am a child!'

In [ ]:
try:
    inst = Child()
except Exception as error:
    print_error(error)

This raises an error when trying to create an instance. This is because the child class needs to override the abstract method `get_id`. 

In [ ]:
class Child(Parent):
    
    def __init__(self):
        self.get_id()
        
    def get_id(self):
        self.whoami = 'I am a child!'
        
inst = Child()
print(inst.whoami)

This allows the parent class, which may be written with no knowledge of the properties of the child class, to impose some conditions on what a given child class should do.

<br>

### Abstract Properties

In the previous notebook we learned how class properties can be defined to impose conditions on the values of certain class attributes. Using Abstract methods we can combine these tools to ensure that a given class has the properties we want.

We start by defining an abstract class with the abstract property `whoami` and its corresponding setter.

In [ ]:
class AstroObject(ABC):
    
    @property
    @abstractmethod
    def whoami(self):
        pass
    
    @whoami.setter
    @abstractmethod
    def whoami(self, value):
        pass

We can then define a child class that inherits this abstract class.

In [ ]:
class Star(AstroObject):
    
    def __init__(self):
        self.whoami = 'I am a star!'

In [ ]:
try:
    star = Star()
except Exception as error:
    print_error(error)

We can see that while a value for the attribute is provided the property is not explicitly defined. This way we can impose that the `Star` class must define a getter and setter for the attribute `whoami`.

In [ ]:
class Star(AstroObject):
    
    def __init__(self):
        self.whoami = 'I am a star!'
    
    @property
    def whoami(self):
        return self._whoami
    
    @whoami.setter
    def whoami(self, value):
        if not isinstance(value, str):
            raise ValueError('whoami must be a string')
        self._whoami = value

star = Star()
print("whoami =", star.whoami)


This in turn could be used in conjunction with what we learned about class composition to ensure that composers have the properties needed to work with the child class.

## 5 Exercises
---

1. Create a parent class and a child class that can identify its progenitor. 
    1. Your parent class should have the class attribute `parent_name` with the value of your choice.
    1. Your child class should have the attribute `name` with the value of your choice.
    1. Printing an instance of your child class should contain its name and its parent's name. *e.g.* 
    
    ```python
    print(Child('Thor'))
    Thor Odinson
    ```

In [ ]:
# Add your solution here

2. Define a class that can be initialised with composer classes that have been constrained by an abstract class.
    1. Define an abstract class called `EarthAttr` that has the abstract method `whatami`, whcih should return a string of your choice.
    1. Define at least two composer classes (*e.g.* `Moon` and `Core`) that satisfy the requirements of `EarthAttr`.
    1. Define a class called `Earth` that composes these classes to get the `whatami` attribute.
    1. Printing an instance of your `Earth` class should include the value of `whatami`. *e.g.*
    
    ```python
    print(Earth(Moon))
    The Earth has a moon!
    ```
    
    5. Finally, define a final composer class (*e.g.* `Lake`) and demonstrate that it will not instantiate if not correctly constrained by `EarthAttr`. You should get the following error:
    
    ```bash
        'Cant instantiate abstract class Lake with abstract methods whatami'
    ```

In [ ]:
# Add your solution here